In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Dropout,BatchNormalization,LocallyConnected2D
from keras.layers import Dense, Activation, Flatten
from keras.utils import to_categorical
from keras import backend as K
import keras

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import scipy
import cv2
import dlib

In [3]:
model = Sequential()

In [4]:
model.add(Conv2D(32, (11, 11), activation='relu', name='C1', input_shape=(152, 152, 3)))

In [5]:
model.add(MaxPooling2D(pool_size=3, strides=2, padding='same', name='M2'))

In [6]:
model.add(Conv2D(16, (9, 9), activation='relu', name='C3'))

In [7]:
model.add(LocallyConnected2D(16, (9, 9), activation='relu', name='L4'))

In [8]:
model.add(LocallyConnected2D(16, (7, 7), strides=2, activation='relu', name='L5') )

In [9]:
model.add(LocallyConnected2D(16, (5, 5), activation='relu', name='L6'))

In [10]:
model.add(Flatten(name='F0'))

In [11]:
model.add(Dense(4096, activation='relu', name='F7'))

In [12]:
model.add(Dropout(rate=0.5, name='D0'))

In [13]:
model.add(Dense(8631, activation='softmax', name='F8'))

In [14]:
deepface_model = keras.models.Model(inputs=model.layers[0].input, outputs=model.layers[-3].output)

In [15]:
model.load_weights("VGGFace2_weights.h5")

In [16]:
def normalize(x):
    return x / np.sqrt(np.sum(np.multiply(x, x)))

In [17]:
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [18]:
def process_img(image_path):
    test_image=tf.keras.preprocessing.image.load_img(image_path,target_size=(152,152))
    test_image=tf.keras.preprocessing.image.img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    return test_image

In [19]:
# name_list = os.listdir('dataset')
# name_list

In [20]:
embedding_list = []
name_list = []
for folderPath in os.listdir('dataset'):
    fpath=os.path.join('dataset',folderPath)
#     print(folderPath)
#     print(fpath)
    for imgpath in os.listdir(fpath):
        imagepath=os.path.join(fpath,imgpath)
        embedding_list.append(deepface_model.predict(process_img(imagepath))[0])
        name_list.append(folderPath)
        print(imagepath)
print(embedding_list)
print(name_list)

1/1 [==============================] - 65s 65s/step
dataset\Avishkar Pawar\Avishkar Pawar_6.jpg
1/1 [==============================] - 0s 67ms/step
dataset\Avishkar Pawar\WhatsApp Image 2022-07-08 at 2.36.22 PM (1).jpeg
1/1 [==============================] - 0s 66ms/step
dataset\Avishkar Pawar\WhatsApp Image 2022-07-08 at 2.36.22 PM.jpeg
1/1 [==============================] - 0s 70ms/step
dataset\Avishkar Pawar\WhatsApp Image 2022-07-08 at 2.36.23 PM (1).jpeg
1/1 [==============================] - 0s 71ms/step
dataset\Avishkar Pawar\WhatsApp Image 2022-07-08 at 2.36.23 PM (2).jpeg
1/1 [==============================] - 0s 70ms/step
dataset\Avishkar Pawar\WhatsApp Image 2022-07-08 at 2.36.23 PM (3).jpeg
1/1 [==============================] - 0s 74ms/step
dataset\Avishkar Pawar\WhatsApp Image 2022-07-08 at 2.36.23 PM (4).jpeg
1/1 [==============================] - 0s 76ms/step
dataset\Avishkar Pawar\WhatsApp Image 2022-07-08 at 2.36.23 PM (5).jpeg
1/1 [==============================] - 0

In [33]:
def predict(image):
    dist_list = []
    R_embedding = deepface_model.predict(process_img(image))[0]
    for emb_db in embedding_list:
        euclidean_distance = findEuclideanDistance(normalize(R_embedding),normalize(emb_db))
        dist_list.append(euclidean_distance)
    min_dist = min(dist_list)
    print(min_dist)
    if min_dist<1.01:
        min_dist_idx = dist_list.index(min_dist)
        name = name_list[min_dist_idx]
        return name
    else:
        name  = 'unkown'
        return name

In [34]:
from pathlib import Path

In [38]:
l=0
r=0
t=0
b=0
detector = dlib.get_frontal_face_detector()
cam = cv2.VideoCapture(0)
while True:
    ret, frame = cam.read()
    
    if not ret:
        print("no img error...")
        break
    
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
    dect_box = detector(img_rgb)

            
    for box in dect_box:
        l=box.left()+8
        r=box.right()-8
        b=box.bottom()
        t=box.top()
        cv2.rectangle(frame, (box.left(), box.top()),(box.right(),box.bottom()), (0,255,0),1)
    
    
    if (t<=0 or b<=0 or l<=0 or r<=0):
        cv2.imshow('video', frame)
        
    elif(not dect_box):
        cv2.imshow('video', frame)


    else:
        crop_img = frame[t:b,l:r]
        Path('Test_face/').mkdir(parents=True, exist_ok = True)
        cv2.imwrite('Test_face/newface.jpg',crop_img)
        R_ImagePath='Test_face/newface.jpg'
        predicted_name = predict(R_ImagePath)
        cv2.putText(frame, predicted_name, (l,t-5),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2,cv2.LINE_AA)
        cv2.imshow('video', frame)
    
    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break

cam.release()
cv2.destroyAllWindows() 

1/1 [==============================] - 0s 74ms/step
0.9027358
1/1 [==============================] - 0s 66ms/step
0.9642529
1/1 [==============================] - 0s 65ms/step
0.944156
1/1 [==============================] - 0s 76ms/step
0.91030014
1/1 [==============================] - 0s 71ms/step
0.8869813
1/1 [==============================] - 0s 65ms/step
0.68794197
1/1 [==============================] - 0s 68ms/step
0.6495392
1/1 [==============================] - 0s 71ms/step
0.7768309
1/1 [==============================] - 0s 69ms/step
0.7796386
1/1 [==============================] - 0s 63ms/step
0.66917264
1/1 [==============================] - 0s 61ms/step
1.0960538
1/1 [==============================] - 0s 63ms/step
0.8720441
1/1 [==============================] - 0s 76ms/step
0.87477326
1/1 [==============================] - 0s 63ms/step
0.84940255
1/1 [==============================] - 0s 63ms/step
0.71752024
1/1 [==============================] - 0s 65ms/step
0.8241141
1/1

In [24]:
cam.release()
cv2.destroyAllWindows() 